### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### Import Packages

In [ ]:
#Install MRI File Managers
!pip install nibabel
!pip install nilearn

In [ ]:
#Import my packages
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/Additional Function Scripts')

import process_nii
import cnn_model

In [ ]:
#Import other Packages
from google.colab import drive

import math
from statistics import mean, stdev
import time
import cv2
import gc
import matplotlib.pyplot as plt
import os
import itertools
from pathlib import Path
import nibabel
import nilearn
from nilearn import image
from nilearn.image import resample_img

import numpy as np
from os import listdir
from os.path import join, basename, isdir
import sklearn.metrics
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle, class_weight
from skimage.color import gray2rgb
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import trange, tqdm, tqdm_notebook
import scipy
import h5py
import zipfile
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot, cm
import matplotlib.pyplot as plt
import matplotlib.image as img
from nibabel.testing import data_path

from numpy.random import seed
seed(2018)
import random
random.seed(2018)
tf.random.set_seed(2018)

import keras
from collections import Counter
from keras.layers import Dense, Input
from keras.models import Model, Sequential
from keras.layers.core import Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D, LSTM, Conv3D, MaxPool3D, Conv1D, MaxPool1D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from keras import initializers, optimizers, losses, regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import PCA, IncrementalPCA
from keras import backend as K
from tensorflow.keras import datasets, layers, models

%matplotlib inline

### Import, Resize, and Create Training Data Sets

In [ ]:
# Visual, Sensorimotor, Salience, Language, FrontoParietal, DorsalAttention, DefaultMode, Cerebellar, 4painBest, 8Best
channel = "8Best"

# 30ICA, 42ICA
ICA_type = "42ICA"

#s2D, VGG16, VGG19, ResNet
model_type = "s2D"

In [ ]:
#Resize Files

#Affine = Detail of images (Higher = less detailed, Lower = more detailed, Normal = 2)
affine_value = 3

%cd /content/gdrive/MyDrive/MRI Data/JamesCNN/{channel}/{ICA_type}/training/Painful
train_painful_scans_resized = np.array([process_nii.resize_5D_to_4D(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])
#train_painful_scans_resized = np.array([process_nii.resize(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])

%cd /content/gdrive/MyDrive/MRI Data/JamesCNN/{channel}/{ICA_type}/training/notPainful
train_painless_scans_resized = np.array([process_nii.resize_5D_to_4D(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])
#train_painless_scans_resized = np.array([process_nii.resize(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])

In [ ]:
#Create Training Data Sets
x_train, y_train = process_nii.create_training_set(train_painful_scans_resized, train_painless_scans_resized)

### Define Hyperparameters

In [ ]:
def load_hyperparameters():

  nEpochs = 50
  batch_size = 16
  callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)
  k_fold_num = 5
  k_fold = StratifiedKFold(n_splits = k_fold_num, shuffle=True)
  METRICS = [
              keras.metrics.BinaryAccuracy(name='accuracy'),
              keras.metrics.AUC(name='auc'),
              keras.metrics.Precision(name='precision'),
              keras.metrics.Recall(name='recall'),
              keras.metrics.TruePositives(name='tp'),
              keras.metrics.FalsePositives(name='fp'),
              keras.metrics.TrueNegatives(name='tn'),
              keras.metrics.FalseNegatives(name='fn'),
            ]
  #class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train), y_train)
  class_weights = {0:0.74166667, 
                   1:1.53448276}

  return nEpochs, batch_size, callback, k_fold, METRICS, class_weights

### Train 2D CNN

In [ ]:
#Display metrics per fold? (LOTS OF OUTPUT!)
view_metrics_per_fold = False

#Load Hyperparameters
nEpochs, batch_size, callback, k_fold, metrics, class_weights = load_hyperparameters()

#Prep result lists
loss_per_fold = []
acc_per_fold = []
auc_per_fold = []
preci_per_fold = []
recall_per_fold = []
tp_per_fold = []
fp_per_fold = []
tn_per_fold = []
fn_per_fold = []
fold_no = 1

print("The shape of the current training set is", x_train.shape,"and contains",y_train.tolist().count(1),"painful and",y_train.tolist().count(0),"painless patients.\n")

for train, test in k_fold.split(x_train, y_train):

  model = cnn_model.s2D(x_train[train], y_train[train], batch_size, nEpochs, callback, metrics, class_weights)
  #model = cnn_model.VGG16(x_train[train], y_train[train], batch_size, nEpochs, callback, metrics, class_weights)
  #model = cnn_model.VGG19(x_train[train], y_train[train], batch_size, nEpochs, callback, metrics, class_weights)
  #model = cnn_model.ResNet(x_train[train], y_train[train], batch_size, nEpochs, callback, metrics, class_weights)

  scores = model.evaluate(x_train[test], y_train[test], verbose=1)
  
  if view_metrics_per_fold:
    print(f'Score for fold {fold_no}:\n{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}; {model.metrics_names[2]} of {scores[2]}; {model.metrics_names[3]} of {scores[3]}; {model.metrics_names[4]} of {scores[4]}; {model.metrics_names[5]} of {scores[5]}; {model.metrics_names[6]} of {scores[6]}; {model.metrics_names[7]} of {scores[7]}; {model.metrics_names[8]} of {scores[8]}')   
  
  loss_per_fold.append(scores[0])
  acc_per_fold.append(scores[1])
  auc_per_fold.append(scores[2])
  preci_per_fold.append(scores[3])
  recall_per_fold.append(scores[4])
  tp_per_fold.append(scores[5])
  fp_per_fold.append(scores[6])
  tn_per_fold.append(scores[7])
  fn_per_fold.append(scores[8])

  fold_no = fold_no + 1

#Print average scores for all folds
print("\n")
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Accuracy: {np.mean(acc_per_fold)} (± {np.std(acc_per_fold)})')
print(f'> AUC: {np.mean(auc_per_fold)} (± {np.std(auc_per_fold)})')
print(f'> PRECISION: {np.mean(preci_per_fold)} (± {np.std(preci_per_fold)})')
print(f'> RECALL: {np.mean(recall_per_fold)} (± {np.std(recall_per_fold)})')
print(f'> TP: {np.mean(tp_per_fold)} (± {np.std(tp_per_fold)})')
print(f'> FP: {np.mean(fp_per_fold)} (± {np.std(fp_per_fold)})')
print(f'> TN: {np.mean(tn_per_fold)} (± {np.std(tn_per_fold)})')
print(f'> FN: {np.mean(fn_per_fold)} (± {np.std(fn_per_fold)})')
print('------------------------------------------------------------------------')

### Save Training Data to Google Drive

In [ ]:
%cd /content/gdrive/MyDrive/MRI Data/JamesCNN/{channel}/{ICA_type}/Results

#Save the model
model.save("{}_CNN_{}_AFFINE_VALUE_{}.h5".format(model_type,ICA_type,affine_value))

#Save metrics for all folds as csv
results = pd.DataFrame(columns=model.metrics_names)
results['loss'] = loss_per_fold
results['accuracy'] = acc_per_fold
results['auc'] = auc_per_fold
results['precision'] = preci_per_fold
results['recall'] = recall_per_fold
results['tp'] = tp_per_fold
results['fp'] = fp_per_fold
results['tn'] = tn_per_fold
results['fn'] = fn_per_fold
results.index += 1 
results.to_csv("TRAIN_METRICS_ALL_FOLDS_{}_CNN_{}_AFFINE_VALUE_{}.csv".format(model_type,ICA_type,affine_value))

#Save average scores (± std) for folds as csv
results = pd.DataFrame(columns=model.metrics_names)
results['loss'] = [np.mean(loss_per_fold), ("± "+ str(np.std(loss_per_fold)))]
results['accuracy'] = [np.mean(acc_per_fold), ("± "+ str(np.std(acc_per_fold)))]
results['auc']= [np.mean(auc_per_fold), ("± "+ str(np.std(auc_per_fold)))]
results['precision'] = [np.mean(preci_per_fold), ("± "+ str(np.std(preci_per_fold)))]
results['recall'] = [np.mean(recall_per_fold), ("± "+ str(np.std(recall_per_fold)))]
results['tp']= [np.mean(tp_per_fold), ("± "+ str(np.std(tp_per_fold)))]
results['fp'] = [np.mean(fp_per_fold), ("± "+ str(np.std(fp_per_fold)))]
results['tn'] = [np.mean(tn_per_fold), ("± "+ str(np.std(tn_per_fold)))]
results['fn'] = [np.mean(fn_per_fold), ("± "+ str(np.std(fn_per_fold)))]
results.index += 1 
results.to_csv("TRAIN_AVERAGE_FOLD_METRICS_{}_CNN_{}_AFFINE_VALUE_{}.csv".format(model_type,ICA_type,affine_value))

### 2D External Validation

In [ ]:
#Resize Files
%cd /content/gdrive/MyDrive/MRI Data/JamesCNN/{channel}/{ICA_type}/testing/Painful
test_painful_scans_resized = np.array([process_nii.resize_5D_to_4D(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])
#test_painful_scans_resized = np.array([process_nii.resize(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])

%cd /content/gdrive/MyDrive/MRI Data/JamesCNN/{channel}/{ICA_type}/testing/notPainful
test_painless_scans_resized = np.array([process_nii.resize_5D_to_4D(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])
#test_painless_scans_resized = np.array([process_nii.resize(file_,affine_value) for file_ in os.listdir() if ".nii" in file_])

#Create Test Data Sets
x_test, y_test = process_nii.create_test_set(test_painful_scans_resized, test_painless_scans_resized)

In [ ]:
#Reset metric lists for each slice
loss_per_fold = []
acc_per_fold = []
auc_per_fold = []
preci_per_fold = []
recall_per_fold = []
tp_per_fold = []
fp_per_fold = []
tn_per_fold = []
fn_per_fold = []

#Load the model in question
%cd /content/gdrive/MyDrive/MRI Data/JamesCNN/{channel}/{ICA_type}/Results
for file_ in os.listdir():
  if ".h5" in file_:
    print("\nLoading",file_,"\n")
    
    model = keras.models.load_model(file_)

    print("The shape of the current testing set is:", x_test.shape,"and contains",y_test.tolist().count(1),"painful and",y_test.tolist().count(0),"painless patients")
    scores = model.evaluate(x_test, y_test, verbose=1)

    loss_per_fold.append(scores[0])
    acc_per_fold.append(scores[1])
    auc_per_fold.append(scores[2])
    preci_per_fold.append(scores[3])
    recall_per_fold.append(scores[4])
    tp_per_fold.append(scores[5])
    fp_per_fold.append(scores[6])
    tn_per_fold.append(scores[7])
    fn_per_fold.append(scores[8])

    #Save metrics for all folds as csv
    results = pd.DataFrame(columns=model.metrics_names)

    results['loss'] = loss_per_fold
    results['accuracy'] = acc_per_fold
    results['auc'] = auc_per_fold
    results['precision'] = preci_per_fold
    results['recall'] = recall_per_fold
    results['tp'] = tp_per_fold
    results['fp'] = fp_per_fold
    results['tn'] = tn_per_fold
    results['fn'] = fn_per_fold
    results['sensitivity'] = tp_per_fold[0] / (tp_per_fold[0] + fn_per_fold[0])
    results['specificity'] = tn_per_fold[0] / (tn_per_fold[0] + fp_per_fold[0])
    results.index += 1 
    results.to_csv("TEST_{}_CNN_AFFINE_VALUE_{}.csv".format(model_type,affine_value))  